In [1]:
import cv2
import mediapipe as mp
import numpy as np
from sklearn.svm import SVC

mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

In [2]:
# Calculating angle and extracting features

def calculate_angle(p1, p2, p3):
    v1 = p1 - p2
    v2 = p3 - p2
    angle_radians = np.arccos(
        np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2))
    )
    angle_degrees = np.degrees(angle_radians)
    return angle_degrees


def extract_features(landmarks):
    left_shoulder = [landmarks[11].x, landmarks[11].y]
    left_elbow = [landmarks[13].x, landmarks[13].y]
    left_wrist = [landmarks[15].x, landmarks[15].y]

    right_shoulder = [landmarks[12].x, landmarks[12].y]
    right_elbow = [landmarks[14].x, landmarks[14].y]
    right_wrist = [landmarks[16].x, landmarks[16].y]

    left_arm_angle = calculate_angle(left_shoulder, left_elbow, left_wrist)
    right_arm_angle = calculate_angle(right_shoulder, right_elbow, right_wrist)

    return [left_arm_angle, right_arm_angle]

In [ ]:
# Load training data

In [ ]:
# Training the model

from sklearn.svm import SVC
from sklearn.model_selection import train_test_split

X, y = load_training_data()

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

classifier = SVC()
classifier.fit(X_train, y_train)

accuracy = classifier.score(X_test, y_test)
print(f"Classifier accuracy: {accuracy}")

In [ ]:
def recognize_punches(cap, classifier):
    with mp_pose.Pose(
        min_detection_confidence=0.5, min_tracking_confidence=0.5
    ) as pose:
        while cap.isOpened():
            ret, image = cap.read()
            if not ret:
                break

            image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            results = pose.process(image_rgb)
            landmarks = results.pose_landmarks.landmark

            if landmarks:
                punch_type = classifier.predict([extracted_features])[0]
                cv2.putText(
                    image,
                    f"Punch: {punch_type}",
                    (10, 30),
                    cv2.FONT_HERSHEY_SIMPLEX,
                    1,
                    (0, 255, 0),
                    2,
                )

                mp_drawing.draw_landmarks(
                    image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS
                )

            cv2.imshow("Punch Recognition", image)

            if cv2.waitKey(1) & 0xFF == ord("q"):
                break

    cap.release()
    cv2.destroyAllWindows()